# Installation

In [ ]:
%%capture
!pip install autora
!pip install autora[all-theorists]

# Imports

In [ ]:
# autora state
from autora.state import StandardState, on_state, Delta

# experiment_runner
from autora.experiment_runner.synthetic.psychophysics.weber_fechner_law import weber_fechner_law

# experimentalist
from autora.experimentalist.grid import grid_pool
from autora.experimentalist.random import random_pool, random_sample

# theorist
from autora.theorist.bms import BMSRegressor


# Creating a State

To create the `state`, we need variable definitions. In this case, we get them from the runner:

In [ ]:
experiment_runner = weber_fechner_law()
experiment_runner.variables

VariableCollection(independent_variables=[IV(name='S1', value_range=(0.01, 5.0), allowed_values=array([0.01      , 0.06040404, 0.11080808, 0.16121212, 0.21161616,
       0.2620202 , 0.31242424, 0.36282828, 0.41323232, 0.46363636,
       0.5140404 , 0.56444444, 0.61484848, 0.66525253, 0.71565657,
       0.76606061, 0.81646465, 0.86686869, 0.91727273, 0.96767677,
       1.01808081, 1.06848485, 1.11888889, 1.16929293, 1.21969697,
       1.27010101, 1.32050505, 1.37090909, 1.42131313, 1.47171717,
       1.52212121, 1.57252525, 1.62292929, 1.67333333, 1.72373737,
       1.77414141, 1.82454545, 1.87494949, 1.92535354, 1.97575758,
       2.02616162, 2.07656566, 2.1269697 , 2.17737374, 2.22777778,
       2.27818182, 2.32858586, 2.3789899 , 2.42939394, 2.47979798,
       2.53020202, 2.58060606, 2.6310101 , 2.68141414, 2.73181818,
       2.78222222, 2.83262626, 2.8830303 , 2.93343434, 2.98383838,
       3.03424242, 3.08464646, 3.13505051, 3.18545455, 3.23585859,
       3.28626263, 3.33666667, 3.

In [ ]:
state = StandardState(variables=experiment_runner.variables)
state

StandardState(variables=VariableCollection(independent_variables=[IV(name='S1', value_range=(0.01, 5.0), allowed_values=array([0.01      , 0.06040404, 0.11080808, 0.16121212, 0.21161616,
       0.2620202 , 0.31242424, 0.36282828, 0.41323232, 0.46363636,
       0.5140404 , 0.56444444, 0.61484848, 0.66525253, 0.71565657,
       0.76606061, 0.81646465, 0.86686869, 0.91727273, 0.96767677,
       1.01808081, 1.06848485, 1.11888889, 1.16929293, 1.21969697,
       1.27010101, 1.32050505, 1.37090909, 1.42131313, 1.47171717,
       1.52212121, 1.57252525, 1.62292929, 1.67333333, 1.72373737,
       1.77414141, 1.82454545, 1.87494949, 1.92535354, 1.97575758,
       2.02616162, 2.07656566, 2.1269697 , 2.17737374, 2.22777778,
       2.27818182, 2.32858586, 2.3789899 , 2.42939394, 2.47979798,
       2.53020202, 2.58060606, 2.6310101 , 2.68141414, 2.73181818,
       2.78222222, 2.83262626, 2.8830303 , 2.93343434, 2.98383838,
       3.03424242, 3.08464646, 3.13505051, 3.18545455, 3.23585859,
       3.

# Running Experimentalists on the State

Experimentalists are AutoRA components that create/sample conditions.

With the variables defined, we can create a pool.

In [ ]:
# the state.conditions are empty for now:
state.conditions

let's create a function that runs on the `state`. The most general way to do this, is to use `on_state` as a wrepper and return a `Delta` object:

In [ ]:
@on_state()
def grid_pool_on_state(variables):
  return Delta(conditions=grid_pool(variables))

The most important thing to keep in mind, when using `on_state` and `Delta` is, that the keyword arguments of the wrapped function and the `Delta` are not arbitrary. They refer to fields of the `state`.

In this case, `grid_pool_on_state` will "grab" the `variables`-field from the state and output into the `conditions`-field of the `state`.

After creating a wrapped function we can run it on the state:

In [ ]:
state = grid_pool_on_state(state)
# the state.conditions are now filled:
state.conditions

,S1,S2
0,0.01,0.010000
1,0.01,0.060404
2,0.01,0.110808
3,0.01,0.161212
4,0.01,0.211616
...,...,...
9995,5.00,4.798384
9996,5.00,4.848788
9997,5.00,4.899192
9998,5.00,4.949596


Let's do try a different pooler that creates a random pool:

In [ ]:
@on_state()
def random_pool_on_state(variables, num_samples, random_state=None):
  return Delta(conditions=random_pool(variables, num_samples, random_state))

Here, the `random_pool` functions expects more arguments: `num_samples` and `random_state`. But these are not fields of the state! When we run this on the state, we will receive an error message:

In [ ]:
state = random_pool_on_state(state)

TypeError: random_pool_on_state() missing 1 required positional argument: 'num_samples'

We need to provide `num_samples`! The reason that we don't need to provide `random_state` is that we defined a default argument when declaring the function.

!!! Warning: You need to provide the keyword in the wrapped function:

In [ ]:
# This still throws an error:
state = random_pool_on_state(state, 5)

TypeError: random_pool_on_state() takes 1 positional argument but 2 were given

In [ ]:
# This works:
state = random_pool_on_state(state, num_samples=10)
state.conditions

,S1,S2
0,4.243939,0.917273
1,1.874949,1.572525
2,4.697576,2.328586
3,3.286263,3.588687
4,1.370909,2.278182
5,4.495960,2.983838
6,1.118889,1.018081
7,0.514040,4.697576
8,0.665253,1.068485
9,3.286263,1.622929


# Practice

## 1. Sampler

Sample 5 conditions from the pool using the `random_sample` function and wrap it:

In [ ]:
@on_state()
def random_sample_on_state(conditions):
  return Delta(conditions=random_sample(...))

In [ ]:
state = grid_pool_on_state(state)
state = random_sample_on_state(...)

## 2. Experiment Data

Create experiment data by running the synthetic runner on stat:



In [ ]:
@on_state()
def run_on_state(conditions):
  data = experiment_runner.run(...)
  return Delta(experiment_data=data)

In [ ]:
state = run_on_state(...)

## 4. Model

Create a model using the BMS regressor.


!!! Note: An "oddity" of autora is, that it expects a object of type list when appending to the models-field

In [ ]:
@on_state()
def bms_on_state(...):
  regressor = BMSRegressor(epochs=10)
  X = ...
  y = ...
  regressor.fit(X, y)
  return Delta(...=[regressor])

In [ ]:
state = bms_on_state(...)

Now all the fields should be filled:

In [ ]:
print('Variables', state.variables)
print('Conditions', state.conditions)
print('Experiment Data', state.experiment_data)
print('Models', state.models)

# Why use State?

After setting up the state-functions, you have a lot of flexibility in using the functions. All of them now work on the state and return a state. You can arbitrarily chain. For example, you can create loops, select specific experimentalists/ runners or theorists on conditions...:

In [ ]:
for i in range(10):
  if i%2:
    state = random_pool_on_state(state, num_samples=5)
  else:
    state = random_pool_on_state(state, num_samples=10)
  state = run_on_state(state, ...)
  state = bms_on_state(state, ...)